<a href="https://colab.research.google.com/github/taeheee-kim/deep_learning_practice/blob/master/%EC%98%A4%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Oct 18 16:53:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    36W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

albumentations==1.1.0 is successfully installed


### Import the required libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve

import albumentations as A
#from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

cudnn.benchmark = True

import gc
import torch
import torch.utils.data as Data
import torch.nn as nn
from torchvision import transforms, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

### Extract csv file from Image files

In [5]:
# Extract csv file from Image files
'''
PATH = '/content/drive/MyDrive/Tomato/train_data/'


file_list = ['Tomato_D01','Tomato_D04','Tomato_D05','Tomato_D07','Tomato_D08','Tomato_D09','Tomato_H','Tomato_P03','Tomato_P05','Tomato_R01'] #폴더 이름 리스트로!

id = [] #이미지명
label = [] #폴더 이름 = Tomato_D01

for file_name in file_list : #D1, R1 10번 

  file_path = os.path.join(PATH,file_name)
  dir_list = os.listdir(file_path)

  for j in range(len(dir_list)):  #1, 4, 6,10 ,100, 2115png
    
    id.append(dir_list[j].replace(".png",""))
    
    label.append(file_name)

df = pd.DataFrame({'file_name':id, 'class':label})
df_shuffled=df.sample(frac=1).reset_index(drop=True) #순서 섞음

df_shuffled
'''

'\nPATH = \'/content/drive/MyDrive/Tomato/train_data/\'\n\n\nfile_list = [\'Tomato_D01\',\'Tomato_D04\',\'Tomato_D05\',\'Tomato_D07\',\'Tomato_D08\',\'Tomato_D09\',\'Tomato_H\',\'Tomato_P03\',\'Tomato_P05\',\'Tomato_R01\'] #폴더 이름 리스트로!\n\nid = [] #이미지명\nlabel = [] #폴더 이름 = Tomato_D01\n\nfor file_name in file_list : #D1, R1 10번 \n\n  file_path = os.path.join(PATH,file_name)\n  dir_list = os.listdir(file_path)\n\n  for j in range(len(dir_list)):  #1, 4, 6,10 ,100, 2115png\n    \n    id.append(dir_list[j].replace(".png",""))\n    \n    label.append(file_name)\n\ndf = pd.DataFrame({\'file_name\':id, \'class\':label})\ndf_shuffled=df.sample(frac=1).reset_index(drop=True) #순서 섞음\n\ndf_shuffled\n'

In [6]:
'''
df = pd.get_dummies(df_shuffled['class'])
id_sh = df_shuffled['file_name']
df.insert(0,"id",id_sh,True)

df.to_csv('/content/drive/MyDrive/Tomato/train.csv') #csv 파일 생성
'''

'\ndf = pd.get_dummies(df_shuffled[\'class\'])\nid_sh = df_shuffled[\'file_name\']\ndf.insert(0,"id",id_sh,True)\n\ndf.to_csv(\'/content/drive/MyDrive/Tomato/train.csv\') #csv 파일 생성\n'

### Python File I/O

In [7]:
data_path = '/content/drive/MyDrive/Tomato/'

train = pd.read_csv(data_path + 'train.csv')

seed = 10
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### Split train and validation dataset

In [8]:
from sklearn.model_selection import train_test_split
import cv2
from torch.utils.data import Dataset  
import numpy as np
import glob

#device 설정 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #device에 cuda 설정

# Create train-validation split
train, valid = train_test_split(train,
                                test_size=0.1,
                                stratify=train.loc[:, 'Tomato_D01':'Tomato_R01'],
                                random_state=10)

### Create a custom dataset class

In [9]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()  
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        img_id = self.df.iloc[idx, 0] 
        img_path = self.img_dir + str(img_id) + '.png'  #str 추가
        image = cv2.imread(img_path) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        
        if self.transform is not None:
            image = self.transform(image=image)['image']

        if self.is_test:
            return image
        else:
            label = np.argmax(self.df.iloc[idx, 1:11]) #타겟 10개중 가장 큰 인덱스 반환
            return image, label

### Define transform function(이미지 증강)

In [10]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
#from albumentations.pytorch import ToTensor

In [11]:
transform_tr = A.Compose([
    A.Resize(400, 600), 
    A.RandomBrightnessContrast(brightness_limit=0.1, 
                               contrast_limit=0.1, p=0.5),
    A.VerticalFlip(p=0.5), 
    A.HorizontalFlip(p=0.5), 

    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=25, p=0.7),

    A.OneOf([A.Emboss(p=1),
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.5),
    A.PiecewiseAffine(p=0.5), 
    A.Normalize(), 
    ToTensorV2() 
    #ToTensor()
])

transform_te = A.Compose([
    A.Resize(400, 600),
    A.Normalize(),
    ToTensorV2()
    #ToTensor()
])

### seed worker 고정

In [12]:
def seed_worker(worker_id): 
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
gen = torch.Generator()
gen.manual_seed(0)

### Data 준비

In [13]:
from torch.utils.data import DataLoader 

batch_size = 6

img_dir = '/content/drive/MyDrive/Tomato/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_tr)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_te)


loader_train = DataLoader(dataset_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=gen)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=gen)

In [14]:
!pip install efficientnet-pytorch==0.7.1

In [15]:
!pip install transformers

### EfficientNet 모델 설정

In [16]:
from efficientnet_pytorch import EfficientNet 

model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10) 

model = model.to(device)

Loaded pretrained weights for efficientnet-b6


### Initialize

In [17]:
import torch.nn as nn 
from transformers import get_cosine_schedule_with_warmup

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00007, weight_decay=0.0001)

epochs = 3

scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(loader_train)*5, 
                                            num_training_steps=len(loader_train)*epochs)


### 모델 학습

In [18]:
train_loss = []
valid_loss = []
train_acc = []
val_acc = []

In [19]:
#df_tl = pd.DataFrame (train_loss)

In [20]:
#path_tl = '/content/drive/My Drive/Tomato/태희/train_loss/tl'+ '.txt'
#df_tl.to_csv(path_tl, sep = '\t')

In [21]:
#print(type(df_tl))

In [22]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대
from sklearn.metrics import accuracy_score


# 총 에폭만큼 반복
for epoch in range(epochs):

    #train
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
    preds_for_acc = [] # 예측값
    labels_for_acc = [] # 실제 값


    # '반복 횟수'만큼 반복 
    for images, labels in tqdm(loader_train):
    
        # 이미지, 레이블(타깃 값) 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)

        # 현재 배치에서의 손실 추가 (훈련 데이터용)
        epoch_train_loss += loss.item() 
        # 영어 코드에서는 loss.item()에 labels.shape[0]을 곱해주는데 이건 왜 그러는지 모르겠넹

        # 역전파 수행
        loss.backward() 
        # 가중치 갱신
        optimizer.step() 
        # 스케줄러 학습률 갱신
        scheduler.step() 

        #영어 코드 복붙
        labels_for_acc = np.concatenate((labels_for_acc, labels.cpu().numpy()),0)
        preds_for_acc = np.concatenate((preds_for_acc, np.argmax(outputs.cpu().detach().numpy(),1)),0)
        
        path = '/content/drive/My Drive/Tomato/태희/epoch_예선_CHECK' + str(epoch) + '.pt'
        torch.save(model.state_dict(), path)

    ta = accuracy_score(labels_for_acc, preds_for_acc)
    tl = epoch_train_loss/len(loader_train)

    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    

    # validation
    
    model.eval() # 모델을 평가 상태로 설정 
    epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용) 
    preds_list = [] # 예측 확률값 저장용 리스트 초기화 
    true_list = [] # 실제 타깃 값 저장용 리스트 초기화 
    

    with torch.no_grad(): # 기울기 계산 비활성
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            true_list = np.concatenate((true_list, labels.cpu().numpy()),0)
            preds_list = np.concatenate((preds_list, np.argmax(outputs.cpu().detach().numpy(),1)),0)

            va = accuracy_score(true_list, preds_list)
            vl = epoch_valid_loss/len(loader_valid)
            
            
        # 검증 데이터 손실값 및 ROC AUC 점수 출력 
        print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')  

    train_loss.append(t1)
    valid_loss.append(v1)
    train_acc.append(ta)
    val_acc.append(va)

    df_tl = pd.DataFrame (train_loss)
    df_vl = pd.DataFrame (valid_loss)
    df_ta = pd.DataFrame (train_acc)
    df_va = pd.DataFrame (val_acc)

    path_tl = '/content/drive/My Drive/Tomato/태희/train_loss/tl' + str(epoch) + '.txt'
    df_tl.to_csv(path_tl, sep = '\t')
    path_vl = '/content/drive/My Drive/Tomato/태희/valid_loss/vl' + str(epoch) + '.txt'
    df_vl.to_csv(path_vl, sep = '\t')
    path_ta = '/content/drive/My Drive/Tomato/태희/train_acc/ta' + str(epoch) + '.txt'
    df_ta.to_csv(path_ta, sep = '\t')
    path_va = '/content/drive/My Drive/Tomato/태희/val_acc/va' + str(epoch) + '.txt'
    df_va.to_csv(path_va, sep = '\t')

  0%|          | 0/2079 [00:00<?, ?it/s]

에폭 [1/3] - 훈련 데이터 손실값 : 1.1050


ValueError: ignored

### Epoch에 따른 Train vs Val 손실값 비교

In [ ]:
plt.figure()
plt.ylim(0,1.5)
sns.lineplot(list(range(len(train_loss))), train_loss)
sns.lineplot(list(range(len(valid_loss))), valid_loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train','Val'])

### Epoch에 따른 Train vs Val 정확도 비교

In [ ]:
plt.figure()
sns.lineplot(list(range(len(train_acc))), train_acc)
sns.lineplot(list(range(len(val_acc))), val_acc)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train','Val'])

### 저장된 모델 불러와서 TTA

In [ ]:
#저장된 Model load
'''
PATH = '/content/drive/MyDrive/Tomato/태희/'

model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10)

device = 'cuda'
model.load_state_dict(copy.deepcopy(torch.load(PATH + "epoch수정12.pt",device))) # 9, 14, 19, 24, 29 

model = model.to(device) 

batch_size = 6
'''

In [ ]:
'''
data_path = '/content/drive/MyDrive/Tomato/'

test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')
'''

In [ ]:
#data loader 설정 및 tta 학습
'''
img_dir = '/content/drive/My Drive/Tomato/test_images/'

dataset_TTA = ImageDataset(test, img_dir=img_dir, 
                           transform=transform_tr, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size, 
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=gen)
'''

In [ ]:
'''
num_TTA = 3

preds_tta = np.zeros((len(test), 10)) 

for i in range(num_TTA):
    print(i+1, "회입니다.")
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)

            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part
            '''

In [ ]:
'''preds_tta /= num_TTA '''

In [ ]:
#submission_csv 파일 저장 및 제출
'''
submission_tta = submission.copy() 

submission_tta[['Tomato_D01','Tomato_D04','Tomato_D05','Tomato_D07','Tomato_D08','Tomato_D09','Tomato_H','Tomato_P03','Tomato_P05','Tomato_R01']] = preds_tta
'''

In [ ]:
'''submission_tta'''

In [ ]:
'''id = submission_tta["file_name"]'''

In [ ]:
'''pred_idx = np.argmax(preds_tta, axis=1)  #확률 중 가장 높은 값의 인덱스를 0 ~ 9까지 반환'''

In [ ]:
#pred_idx

In [ ]:
#df = pd.DataFrame({'file_name':id, 'answer':pred_idx})

In [ ]:
#df 

In [ ]:
#df.to_csv('/content/drive/MyDrive/Tomato/현중/최종.csv', index=False)